<a href="https://colab.research.google.com/github/williambrunos/Deep-Learning-Neuro-evolution/blob/main/Fundamentals/evolving_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evolving Neural Networks

## Abstract

Evlutionary algorithms are way used to perform topologies, weights, biases and neurons evolutions across neural networks. Sometimes, it's way more pragmatic than other ML algorithms and have to be set to some specific problem.

## Algorithm

Evolutionary algorithms are based on the premisse of **natural selection**, based on the following steps:

- The algorithm starts a population with a certain number of individuals/genomes, with their genotypes or carachteristics being initiated randomly.
- The organisms are evaluated based on a **fitness function** or **fitness score**.
- The best individuals prom the population are choiced to be parents, performing **asexual reprodution** if the offspring has the same genotype as the parents, or **sexual reprodution** if the offspring has the genotype being a combination of the genotypes of the parents.
- Mutate the offspring.
- Take back to step two and iterates over these steps until some condition is met, being a maximun number of steps, a threshold of the fitness score etc.

## Step 1: The Algorithm

We'll be using dense (fully-connected) feed-foward neural networks, like in figure 1:

![NN image](https://miro.medium.com/max/700/1*Thii4-1bSrJ1yXwDyW3sjw.png
)

<center>Dense NN with dimensions [1, 12, 12, 12, 1]</center>

In designing our organisms, we have four guiding principles:

- We must have control over the input and output dimensions of the organisms. Fundamentally, we are trying to evolve some function f that maps ℝᵃ ⟶ ℝᵇ, so a and b should be built into the organism. We accomplish this by parameterizing the input and output dimensions.
- We must have control over the output activation function. The output of the organisms should be appropriate for the problem at hand. We accomplish this by parameterizing the output activation.
- We must have control over the complexity of the organisms. The ideal organism should be just complex enough to evolve the target function and no more. We accomplish this by parameterizing the number of hidden layers and their dimensions. In a more advanced algorithm, this could be achieved by letting the organism evolve its own architecture and penalizing complexity with the fitness function.
- Organisms must be compatible for sexual reproduction. Fortunately, the above principles ensure that this will be the case. All organisms will have the same architecture, so “exchanging genetic material” here means offspring will get some layer weights from momma and some from poppa.

## Refferences

[Mediun article - Evolving Neural Networks](https://towardsdatascience.com/evolving-neural-networks-b24517bb3701)